Before you turn in the homework, make sure everything runs as expected. To do so, select **Kernel**$\rightarrow$**Restart & Run All** in the toolbar above.  Remember to submit both on **DataHub** and **Gradescope**.

Please fill in your name and include a list of your collaborators below.

In [1]:
NAME = "Vinay Maruri"
COLLABORATORS = ""

---

# Project 2: NYC Taxi Rides
# Extras

Put all of your extra work in here. Feel free to save figures to use when completing Part 4.

In [2]:
import os
import pandas as pd
import numpy as np
import sklearn.linear_model as lm
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sqlalchemy import create_engine
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

sns.set(style="whitegrid", palette="muted")

plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.size'] = 12

%matplotlib inline

In [3]:
from pylab import savefig

In [4]:
# Run this cell to load the data. 
data_file = Path("./", "cleaned_data.hdf")
train_df = pd.read_hdf(data_file, "train")
val_df = pd.read_hdf(data_file, "val")

In [5]:
train_df.columns.values

array(['record_id', 'VendorID', 'tpep_pickup_datetime',
       'tpep_dropoff_datetime', 'passenger_count', 'trip_distance',
       'pickup_longitude', 'pickup_latitude', 'RatecodeID',
       'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude',
       'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'total_amount', 'duration'], dtype=object)

In [9]:
train_df

,record_id,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,...,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,duration
13242,5711100,1,2016-01-17 17:48:41,2016-01-17 17:55:53,1,1.00,-74.006470,40.738766,1,N,...,40.735664,1,6.5,0.0,0.5,1.45,0.00,0.3,8.75,432.0
12723,4989400,1,2016-01-17 01:18:39,2016-01-17 01:21:15,1,0.40,-73.989365,40.763000,1,N,...,40.766121,2,4.0,0.5,0.5,0.00,0.00,0.3,5.30,156.0
8508,2436400,2,2016-01-12 09:07:00,2016-01-12 09:41:17,1,11.40,-73.984108,40.774509,1,N,...,40.770458,1,37.0,0.0,0.5,8.67,5.54,0.3,52.01,2057.0
21304,10899100,2,2016-01-29 09:07:54,2016-01-29 09:18:25,1,1.42,-74.002907,40.760262,1,N,...,40.742764,1,8.5,0.0,0.5,1.00,0.00,0.3,10.30,631.0
3817,1319400,1,2016-01-06 11:44:54,2016-01-06 11:49:55,1,0.80,-73.969742,40.760273,1,N,...,40.751129,2,5.0,0.0,0.5,0.00,0.00,0.3,5.80,301.0
5503,10377300,2,2016-01-08 16:46:10,2016-01-08 16:50:53,1,0.98,-73.980614,40.745098,1,N,...,40.752670,1,5.5,1.0,0.5,1.46,0.00,0.3,8.76,283.0
21533,7033400,2,2016-01-29 15:00:37,2016-01-29 15:08:34,1,0.93,-73.980194,40.751087,1,N,...,40.742603,1,7.0,0.0,0.5,0.00,0.00,0.3,7.80,477.0
5959,10487600,2,2016-01-09 01:55:29,2016-01-09 02:10:53,1,3.65,-73.992393,40.725224,1,N,...,40.693737,1,15.0,0.5,0.5,3.26,0.00,0.3,19.56,924.0
16709,8738800,2,2016-01-21 22:28:17,2016-01-21 22:40:33,1,2.67,-73.997986,40.741215,1,N,...,40.709789,1,11.0,0.5,0.5,2.46,0.00,0.3,14.76,736.0
4487,1470800,1,2016-01-07 09:56:15,2016-01-07 10:06:45,1,1.60,-73.969833,40.784542,1,N,...,40.792049,1,9.0,0.0,0.5,1.95,0.00,0.3,11.75,630.0


In [7]:
val_df.columns.values

array(['record_id', 'VendorID', 'tpep_pickup_datetime',
       'tpep_dropoff_datetime', 'passenger_count', 'trip_distance',
       'pickup_longitude', 'pickup_latitude', 'RatecodeID',
       'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude',
       'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'total_amount', 'duration'], dtype=object)

In [8]:
val_df['mta_tax']

6786     0.5
10800    0.5
14240    0.5
6130     0.5
20062    0.5
16681    0.5
15337    0.5
6891     0.5
18046    0.5
12930    0.5
10810    0.5
12036    0.5
6605     0.5
139      0.5
13179    0.5
6124     0.5
20754    0.5
12948    0.5
17835    0.5
14178    0.5
10950    0.5
5481     0.5
5371     0.5
2742     0.5
21129    0.5
9608     0.5
12035    0.5
6931     0.5
17762    0.5
4106     0.5
        ... 
11706    0.5
12709    0.5
18243    0.5
6099     0.5
7428     0.5
4876     0.5
20241    0.5
15545    0.5
17571    0.5
10344    0.5
7853     0.5
8291     0.5
7849     0.5
12248    0.5
18376    0.5
21900    0.5
20358    0.5
6122     0.5
3261     0.5
12360    0.5
14322    0.5
21218    0.5
13539    0.5
18044    0.5
6396     0.5
20223    0.5
16628    0.5
14357    0.5
586      0.5
14713    0.5
Name: mta_tax, Length: 4431, dtype: float64

In [9]:
# Copied from part 2
def haversine(lat1, lng1, lat2, lng2):
    """
    Compute haversine distance
    """
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    average_earth_radius = 6371
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * average_earth_radius * np.arcsin(np.sqrt(d))
    return h

# Copied from part 2
def manhattan_distance(lat1, lng1, lat2, lng2):
    """
    Compute Manhattan distance
    """
    a = haversine(lat1, lng1, lat1, lng2)
    b = haversine(lat1, lng1, lat2, lng1)
    return a + b

# Copied from part 2
def bearing(lat1, lng1, lat2, lng2):
    """
    Compute the bearing, or angle, from (lat1, lng1) to (lat2, lng2).
    A bearing of 0 refers to a NORTH orientation.
    """
    lng_delta_rad = np.radians(lng2 - lng1)
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    y = np.sin(lng_delta_rad) * np.cos(lat2)
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(lng_delta_rad)
    return np.degrees(np.arctan2(y, x))

# Copied from part 2
def add_time_columns(df):
    """
    Add temporal features to df
    """
    df.is_copy = False # propogate write to original dataframe
    df.loc[:, 'month'] = df['tpep_pickup_datetime'].dt.month
    df.loc[:, 'week_of_year'] = df['tpep_pickup_datetime'].dt.weekofyear
    df.loc[:, 'day_of_month'] = df['tpep_pickup_datetime'].dt.day
    df.loc[:, 'day_of_week'] = df['tpep_pickup_datetime'].dt.dayofweek
    df.loc[:, 'hour'] = df['tpep_pickup_datetime'].dt.hour
    df.loc[:, 'week_hour'] = df['tpep_pickup_datetime'].dt.weekday * 24 + df['hour']
    return df

# Copied from part 2
def add_distance_columns(df):
    """
    Add distance features to df
    """
    df.is_copy = False # propogate write to original dataframe

    df.loc[:, 'bearing'] = bearing(lat1=df['pickup_latitude'],
                                   lng1=df['pickup_longitude'],
                                   lat2=df['dropoff_latitude'],
                                   lng2=df['dropoff_longitude'])
    df.loc[:, 'haversine'] = haversine(lat1=df['pickup_latitude'],
                                   lng1=df['pickup_longitude'],
                                   lat2=df['dropoff_latitude'],
                                   lng2=df['dropoff_longitude'])
    df.loc[:, 'manhattan'] = manhattan_distance(lat1=df['pickup_latitude'],
                                                lng1=df['pickup_longitude'],
                                                lat2=df['dropoff_latitude'],
                                                lng2=df['dropoff_longitude'])
    return df

def select_columns(data, *columns):
    return data.loc[:, columns]

In [50]:
def process_data_gm2(data, test=False):
    X = (
        data
        
        # Transform data
        .pipe(add_time_columns)
        .pipe(add_distance_columns)
        .pipe(select_columns,        
              'pickup_longitude',  
              'pickup_latitude',   
              'dropoff_longitude', 
              'dropoff_latitude',
              'fare_amount',
              'trip_distance', 
              'passenger_count', 
              'tip_amount', 
              'tolls_amount',
              'manhattan', 
              'total_amount'
             )
    )
    if test:
        y = None
    else:
        y = data['duration']
        
    return X, y

In [11]:
#My modified Processing pipeline.
#I am adding fare_amount, trip_distance, as there seems to be a correlation 
#between these two features and duration, which is what we are predicting.

In [ ]:
ax = sns.distplot(a = train_df['fare_amount'])
plt.ylabel('Proportion')
plt.title('Distribution of total ride amounts in the dataset')
ax = ax.get_figure()
ax.savefig('ride_hist.png', dpi = 400)

In [ ]:
ax = sns.lineplot(x = 'fare_amount', y = 'duration', data = train_df)
ax = ax.get_figure()
ax.savefig('fareline.png', dpi = 400)

In [ ]:
ax = sns.distplot(a = train_df['trip_distance'])
plt.ylabel('Proportion')
plt.title('A distribution of trip distances in the dataset.')
ax = ax.get_figure()
ax.savefig('trip_hist.png', dpi = 400)

In [ ]:
ax = sns.lineplot(x = 'trip_distance', y = 'duration', data = train_df)
ax = ax.get_figure()
ax.savefig('tripline.png', dpi = 400)

In [51]:
# Train
X_train, y_train = process_data_gm2(train_df)
"""#X_train = X_train.replace(to_replace = 0.0, value = 
               {'pickup_longitude': np.median(X_train['pickup_longitude']), 
                'pickup_latitude': np.median(X_train['pickup_latitude']), 
                'dropoff_longitude': np.median(X_train['dropoff_longitude']), 
                'dropoff_latitude': np.median(X_train['dropoff_latitude'])
               })"""
X_val, y_val = process_data_gm2(val_df)
"""#X_val = X_val.replace(to_replace = 0.0, value = 
               {'pickup_longitude': np.median(X_val['pickup_longitude']), 
                'pickup_latitude': np.median(X_val['pickup_latitude']), 
                'dropoff_longitude': np.median(X_val['dropoff_longitude']), 
                'dropoff_latitude': np.median(X_val['dropoff_latitude'])
               })"""
guided_model_2 = lm.LinearRegression(fit_intercept=True)
guided_model_2.fit(X_train, y_train)

# Predict
y_train_pred = guided_model_2.predict(X_train)
y_val_pred = guided_model_2.predict(X_val)

/srv/conda/envs/data100/lib/python3.6/site-packages/pandas/core/generic.py:4388: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  object.__getattribute__(self, name)
/srv/conda/envs/data100/lib/python3.6/site-packages/pandas/core/generic.py:4389: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  return object.__setattr__(self, name, value)


In [52]:
y_train_pred[y_train_pred < 0] = 0
y_val_pred[y_val_pred < 0] = 0

In [19]:
X_val.isnull().any()

fare_amount        False
trip_distance      False
passenger_count    False
tip_amount         False
tolls_amount       False
manhattan          False
total_amount       False
dtype: bool

In [20]:
X_train.isnull().any()

fare_amount        False
trip_distance      False
passenger_count    False
tip_amount         False
tolls_amount       False
manhattan          False
total_amount       False
dtype: bool

In [39]:
assert 600 <= np.median(y_train_pred) <= 700
assert 600 <= np.median(y_val_pred) <= 700

In [53]:
X_train.iloc[0, :].dot(guided_model_2.coef_) + guided_model_2.intercept_

473.41016929710167

In [54]:
y_train_pred[0]

473.41016929701436

In [55]:
def mae(actual, predicted):
    """
    Calculates MAE from actual and predicted values
    Input:
      actual (1D array-like): vector of actual values
      predicted (1D array-like): vector of predicted/fitted values
    Output:
      a float, the MAE
    """
    
    mae = np.mean(np.abs(actual - predicted))
    return mae

In [56]:
print("Validation Error: ", mae(y_val_pred, y_val))

Validation Error:  147.954972584


In [57]:
print("Training Error: ", mae(y_train_pred, y_train))

Training Error:  147.343071359


These appear to be markedly lower validation and training errors than the guided example in the part 4 notebook.

In [ ]:
# Distribution of residuals
plt.figure(figsize=(8,4))
ax = sns.distplot(np.abs(y_val - y_val_pred))
plt.xlabel('residual')
plt.title('distribution of residuals')
ax = ax.get_figure()
ax.savefig('residual_hist.png', dpi = 400)

In [ ]:
# Average residual grouping by ride duration
val_residual = X_val.copy()
val_residual['duration'] = y_val
val_residual['rounded_duration'] = np.around(y_val, -2)
val_residual['residual'] = np.abs(y_val - y_val_pred)
tmp = val_residual.groupby('rounded_duration').mean()
plt.figure(figsize=(8,4))
tmp['residual'].plot()
plt.ylabel('average residual')
plt.title('average residual by ride duration')

In [ ]:
ax = sns.lineplot(x = 'passenger_count', y = 'duration', data = train_df)
ax = ax.get_figure()
ax.savefig('passline.png', dpi = 400)

In [ ]:
ax = sns.lineplot(x = 'tip_amount', y = 'duration', data = train_df)
ax = ax.get_figure()
ax.savefig('tipline.png', dpi = 400)

In [ ]:
ax = sns.lineplot(x = 'tolls_amount', y = 'duration', data = train_df)
ax = ax.get_figure()
ax.savefig('tollline.png', dpi = 400)

In [ ]:
ax = sns.lineplot(x = 'total_amount', y = 'duration', data = train_df)
ax = ax.get_figure()
ax.savefig('totalline.png', dpi = 400)

## Submission

You're almost done!

Before submitting this assignment, ensure that you have:

1. Restarted the Kernel (in the menubar, select Kernel$\rightarrow$Restart & Run All)
2. Validated the notebook by clicking the "Validate" button.

Then,

1. **Submit** the assignment via the Assignments tab in **Datahub** 
1. **Upload and tag** the manually reviewed portions of the assignment on **Gradescope**